In [ ]:
import numpy as np
# test area
np.maximum.accumulate([1,2,3,1,5])
np.power(2,3)

# Load Data

In [ ]:
%matplotlib inline
import pandas
import numpy as np
from matplotlib import pyplot as plt

df = pandas.read_csv('d.csv', index_col=0)
names = df.index
columns = df.columns

a = df.iloc[:, :-1]
b = df.iloc[:, 1:]
b.columns = a.columns
r = (b - a) / a
r = r.values
base = 10
df = pandas.DataFrame(np.concatenate((np.ones([r.shape[0], 1]) * base,
                np.add.accumulate(r * base, axis=1) + np.ones(r.shape) * base), axis=1))
df.index = names
df.columns = columns

origin = df.copy()
for i in range(2, 11):
    n = origin.copy() * i
    n.index = map(lambda x: str(i) + x, origin.index)
    df = pandas.concat([df, n])

l = df.shape[0]
correlation = np.ones([l, l])




d = df.values
df.index[[7, 7+71]]

In [ ]:
V = np.array([list(d[6]), list(d[64])])

import numpy as np
import tensorflow as tf
import time
import json


w = tf.get_variable("weight", [V.shape[0], 1], tf.float32, initializer=tf.truncated_normal_initializer)
a = tf.get_variable("a", [1], tf.float32, initializer=tf.truncated_normal_initializer)
b = tf.constant(10, tf.float32)


x = np.arange(V.shape[1])
y = a * x + b

v = tf.constant(V, tf.float32)
s = tf.reduce_sum(w * v, axis=0)

loss = tf.reduce_sum(tf.pow(s - y, 2))
total_loss = loss

optimizer = tf.train.AdamOptimizer(1)
train_op = optimizer.minimize(total_loss)

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)

    for step in range(10000000):
        sess.run(train_op)
        if step % 3000 == 0:
            print(step)
            print(sess.run(a))
            print(sess.run(w))
            print(sess.run(s))
            print(sess.run(y))
            print(sess.run(total_loss))

In [ ]:
w = np.array([[0.27],[0.66]])
s = (w * V).sum(axis=0)
s = s / s[0] * 10
graph = np.concatenate([V, [s]], axis=0).T
pandas.DataFrame(graph).plot()
pandas.DataFrame(d[64]).plot()

In [ ]:
latest = np.load('lasttime.npy')
print(latest.shape)
print(latest[0:3])
print(latest[-3:])

dict = {}
for a in latest:
    for b in a[0]:
        if b in dict:
            dict[b] += 1
        else:
            dict[b] = 1
x = []
for key in dict:
    x.append([key, dict[key]])
x = sorted(x, key=lambda m: m[1], reverse=True)

for y in x[:500]:
    print(df.index[y[0]], y[0], y[1])


for a in latest:
    if 6 in a[0]:
        print(list(df.index[a[0]]), a)
    
for a in latest[0:10]:
    print(list(df.index[a[0]]), a)

idx = latest[0][0]

In [ ]:
for a in latest:
    s = (d[a[0]]).sum(axis=0)
    drop = 1 - s / np.maximum.accumulate(s)
    drop_i = np.argsort(drop)
    def getIJ(ij, i):
        return ij[i], np.argmax(s[:ij[i]])

    i, j = getIJ(drop_i, -1)     
    if drop[i] < 0.01:
        print('max drawback {:0.2f}% {:0.2f} {:0.2f} {} {}'.format(drop[i] * 100, s[i], s[j], i, j))
        print('average drop', drop.mean())
        print(list(df.index[a[0]]), a)

# Test By tiger data

In [ ]:
idx = [6, 166, 166, 341, 466, 705]
columns = list(np.array(df.index)[idx])
print(columns)

dates = df.columns.astype(np.float32)
startDate = dates[0]
dates = np.round((dates - startDate) / 3600 / 24 / 1000 / 7) + 1



# basic info
a = d[idx][:, 1:]
b = d[idx][:, :-1]
r = a - b
contri= np.abs(r).mean(axis=1)
contri = contri / contri.min()
print("contribution: {}".format(contri))

graph = pandas.DataFrame(d[idx]).T
graph.columns = columns 
graph.plot()

s = (d[idx]).sum(axis=0)
def evaluate(s, dates, components):
    # regression 
    s = s / len(idx)
    A, B = np.polyfit(dates, np.log(s), 1)
    print('A,B', A, B)
    y = np.exp(A * dates + B)
    last = s[0] < y[0]
    reg = 0
    for i in range(1, len(s)):
        if (s[i] < y[i]) != last:
            reg += 1
            last = s[i] < y[i]
    print('revisit times: ', reg)
    
    loss = (((y - s) / s) ** 2)
    print('loss: ', np.array(loss).sum())
    print(y[-1] / y[0] - 1)


    # max drawback 
    drop = 1 - s / np.maximum.accumulate(s)
    drop_i = np.argsort(drop)
    def getIJ(ij, i):
        return ij[i], np.argmax(s[:ij[i]])

    i, j = getIJ(drop_i, -1)     
    print('max drawback {:0.2f}% {:0.2f} {:0.2f} {} {}'.format(drop[i] * 100, s[i], s[j], i, j))
    print('average drop', drop.mean())
    pandas.DataFrame(drop).hist(bins=20)


    print(s[-1])
    print(dates[-1])

    plt.figure()
    plt.plot(dates, y)
    plt.plot(dates, s)
    plt.plot([i+1,j+1], [s[i], s[j]], 'o', color='Red', markersize=4)

    plt.figure(figsize=(9,25))
    plt.plot(dates, y)
    plt.plot(dates, s)
    if components is not None:
        plots = []
        for x in range(len(components.values.T)):
            plots.append(plt.plot(dates, components.values.T[x], label=components.columns[x])[0])
        plt.legend(handles=plots)
    plt.plot([i+1,j+1], [s[i], s[j]], 'o', color='Red', markersize=4)

    i2, j2 = getIJ(drop_i, -2)
    plt.plot([i2+1,j2+1], [s[i2], s[j2]], 'o', color='Blue', markersize=4)

evaluate(s, dates, graph)

In [ ]:
columns = list(np.array(df.index)[idx])
print(columns)
names = list(map(lambda c: c.split('#')[1], columns))

df2 = None
for n in names:
    f = '/Users/junnwang/Downloads/csv 2/' + n + '_week.csv'
    if df2 is None:
        df2 = pandas.read_csv(f, index_col=0).T
    else:
        df2 = pandas.concat([df2,pandas.read_csv(f, index_col=0).T], axis=1)
print(df2.dropna(axis=0,how='any').astype('float32').shape)     
df2 = df2.iloc[-160:,:]
d2 = df2.dropna(axis=1,how='any').astype('float32').T
a = d2.iloc[:, :-1]
b = d2.iloc[:, 1:]
print(a.shape)
print(b.shape)
b.columns = a.columns = range(d2.shape[1] - 1)
r = (b - a) / a
p2 = p[idx]
r = (r.T * p2).T.values


dates = d2.columns.astype(np.float32)
startDate = dates[0]
dates = np.round((dates - startDate) / 3600 / 24 / 1000 / 7) + 1

dd2 = np.concatenate((np.ones([r.shape[0], 1]) * base,
                        np.add.accumulate(r, axis=1) + np.ones(r.shape) * base), axis=1)

s = dd2.sum(axis=0)
print("percentage of total value: {}".format(p2 / p2.sum() * 100))

dd2 = pandas.DataFrame(dd2.T)
dd2.columns = columns
evaluate(s, dates, dd2)

# Test by yahoo data

In [ ]:
from glob import glob
df = None
for f in glob('sss\\*.csv'):
    name = f.split('\\')[1].split('.')[0]
    if df is None:
        df = pandas.read_csv(f, index_col=0)[['Adj Close']]
        df.columns = [name]
    else:
        csv = pandas.read_csv(f, index_col=0)[['Adj Close']]
        csv.columns = [name]
        df = pandas.concat([df,csv], axis=1)
        
df = df.dropna().T
print(df.shape)
a = df.iloc[:, :-1]
b = df.iloc[:, 1:]
b.columns = a.columns
r = (((b - a) / a).T * p[idx] ).T
r = np.concatenate((np.ones([r.shape[0], 1]) * base,
                    np.add.accumulate(r, axis=1) + np.ones(r.shape) * base), axis=1).T
r = pandas.DataFrame(r)
r.index = df.columns.astype(np.datetime64)
r.columns = df.index
r.plot(figsize=(16, 9))
print(r.shape)

y2 = r.sum(axis=1)
#plt.figure(figsize=(16, 9))
plt.plot(df.columns.astype(np.datetime64), y2 / len(idx))


# Test random output distribution

In [ ]:
import random
import dao

dist = []
n = 6
for i in range(500000):
    choice = random.sample(range(d.shape[0]), n)
#     bad = True
#     while bad:
#         bad = False
#         for k in range(len(choice)):
#             for l in range(k):
#                 if np.abs(correlation[choice[k]][choice[l]]) == 0:
#                     bad = True
#                     break
#             if bad:
#                 break
#         if bad:
#             choice = random.sample(range(d.shape[0]), n)

    result = dao.test_choice(choice)
    dist.append(result[1])
    
pandas.DataFrame(dist).hist(bins=1000)
pandas.qcut(dist,  [0, 0.05, 0.1, 0.25, 0.5, 0.75, 0.999, 1])

# Load Weekly data and clean into d.csv

In [ ]:
%matplotlib inline
import pandas
from glob import glob
import numpy as np

search = ['SPY','EEM','QQQ','HYG','IWM','XLF','VXX','UVXY','EFA','FXI','EWZ','TLT','GLD','FEZ','SMH','USO','SLV','GDX','XLI','XLE','DIA','XRT','XOP','KRE','AAPL','BAC','BABA','TSLA','FB','AMZN','GE','AMD','MU','C','INTC','AABA','EA','JPM','F','NXPI','NFLX','TWTR','MSFT','FOXA','SNAP','WYNN','GM','CAT','WFC','X','T','JD','PBR','QCOM','USB','AA','WMT','DBX','AAL','BA','CMCSA','BIDU','KMI','MS','MET','FCX','NVDA','M','SQ','GS','V','BK','DB']

df = None
# for f in glob('D:\\work\\main\\IB\\csv\\*_week.csv'):
#     if df is None:
#         df = pandas.read_csv(f, index_col=0).T
#     else:
#         df = pandas.concat([df,pandas.read_csv(f, index_col=0).T], axis=1)
for s in search:
    f = 'D:\\work\\main\\IB\\csv\\' + s + '_week.csv'
    if df is None:
        df = pandas.read_csv(f, index_col=0).T
    else:
        df = pandas.concat([df,pandas.read_csv(f, index_col=0).T], axis=1)



d = df.iloc[-112:-1,:]
d = d.dropna(axis=1,how='any').astype('float32').T
print(d.shape)
d.to_csv('d.csv')

# Load Daily data

In [ ]:
from glob import glob
%matplotlib inline
import pandas
import numpy as np
from matplotlib import pyplot as plt

daily = None
for f in glob('D:\\work\\main\\IB\\*_day.csv'):
    if daily is None:
        daily = pandas.read_csv(f, index_col=0).T
    else:
        daily = pandas.concat([daily,pandas.read_csv(f, index_col=0).T], axis=1)
daily.to_csv('day.csv')

# Test by day data

In [ ]:
from glob import glob
daily = pandas.read_csv('day.csv', index_col=0)[columns].T
print(columns)
moneyNeeded = 100 / (p[idx]/p[idx].sum()/daily.T.iloc[-1,:].values)[2]
tobuy = np.round(moneyNeeded * p[idx]/p[idx].sum() /  daily.T.iloc[-1,:].values)
moneyNeeded = tobuy * daily.T.iloc[-1,:].values
print('shares ', str(tobuy), tobuy.sum(), moneyNeeded)
moneyNeeded = moneyNeeded.sum()
print(moneyNeeded)

a = daily.iloc[:, :-1]
b = daily.iloc[:, 1:]
a.columns = b.columns
r = ((b / a - 1).T * p[idx] ).T
print(r.shape)

dates = daily.columns.astype(np.float32)
dates = np.round((dates - startDate) / 3600 / 24 / 1000) / 7 + 1
base = 10.4
r = np.concatenate((np.ones([r.shape[0], 1]) * base,
                    np.add.accumulate(r, axis=1) + np.ones(r.shape) * base), axis=1).T
r = pandas.DataFrame(r)
r.columns = columns
r.index = dates
r.plot(figsize=(16,9))

y = np.exp(A * dates + B) * moneyNeeded / 10
y2 = r.sum(axis=1) / len(columns) * moneyNeeded / 10
plt.figure(figsize=(16,9))
plt.plot(dates, y)
plt.plot(dates, y2)
#plt.plot(111, np.exp(A * 111 + B), marker='o', color='r', ls='')
#y2[-1] / y[-1] - 1